# **TAREA LanceDB**
- Considera usar ANN para cada búsqueda o filtro

**Task 1: Consulta avanzada con proyección y filtro**

Instrucciones:
1. Genera un vector aleatorio con la misma dirección que los embeddings que están en la tabla mis_vectores
2. Realiza una búsqueda en la tabla para encontrar los 5 elementos más cercanos
3. Proyecta los resultados para mostrar solo las columnas item y _distance
4. Excluye de los resultado los elementos cuyo nombre sea 'item 500'

Pregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?

In [14]:
import lancedb
import numpy as np


uri = "/Users/juancasasmartinez/Documents/ITAM/Fuentes/proyectos_finales/proyecto_lance/notebooks/data/ANN"
db = lancedb.connect(uri)
table = db.open_table("mis_vectores")

len(table.to_pandas().iloc[0]['vector'])

1536

In [23]:
# 1.
query_vector = np.random.randn(1536)
query_vector = query_vector / np.linalg.norm(query_vector)  

# 2. 3. 4.
results = table.search(query_vector).where("item != 'item 500'").limit(5).to_pandas()[['item', '_distance']]


print("\nLos 5 elementos más cercanos y sus distancias son:")
for _, row in results.iterrows():
    print(f"{row['item'].capitalize()}, Distancia: {row['_distance']:.4f}")


Los 5 elementos más cercanos y sus distancias son:
Item 91754, Distancia: 414.7281
Item 63165, Distancia: 415.0065
Item 119503, Distancia: 415.8102
Item 188292, Distancia: 417.5602
Item 163104, Distancia: 418.6127


**Task 2: Creación de tablas**

Instrucciones:
1. Define un nuevo esquema para una tabla vacía con las siguientes columnas:
*   vector (vector de 4 dimensiones)
*   nombre
*   categoria
2. Crea una tabla vacía llamada nueva_tabla usando el esquema
3. Inserta 5 registros en la tabla
4. Muestra el contenido de la tabla





In [26]:
import pandas as pd
import pyarrow as pa

data = []

for i in range(5):
    vector = np.random.rand(4).astype('float32')
    
    data.append({
        'vector': vector.tolist(),
        'nombre': f'imagen_{i+1}.jpg',
        'categoria': np.random.choice(['paisaje', 'retrato', 'naturaleza'])
    })

df = pd.DataFrame(data)


# 1. 
schema = pa.schema([
    ("vector", pa.list_(pa.float32(), list_size=4)),
    ("nombre", pa.string()),
    ("categoria", pa.string())
])

# 2. 
table = pa.Table.from_pandas(df, schema=schema)
nueva_tabla = db.create_table("nueva_tabla", data=table, mode="overwrite")

nueva_tabla.to_pandas()


[2024-12-03T00:37:14Z WARN  lance::dataset] No existing dataset at /Users/juancasasmartinez/Documents/ITAM/Fuentes/proyectos_finales/proyecto_lance/notebooks/data/ANN/nueva_tabla.lance, it will be created


,vector,nombre,categoria
0,"[0.8146985, 0.42480254, 0.67282945, 0.008298699]",imagen_1.jpg,retrato
1,"[0.61230177, 0.9474902, 0.93997717, 0.6534582]",imagen_2.jpg,naturaleza
2,"[0.91663516, 0.84984857, 0.5960665, 0.78153455]",imagen_3.jpg,naturaleza
3,"[0.23287244, 0.16927856, 0.2698172, 0.6164252]",imagen_4.jpg,retrato
4,"[0.87131745, 0.5638662, 0.15084295, 0.010460604]",imagen_5.jpg,paisaje


**Task 3: Actualización de vectores y filtrado**

Instrucciones:
1. Crea una tabla utilizando un DataFrame de Pandas con las siguientes columnas:
- id (Entero).
- vector (Lista de tres números flotantes)
2. Actualiza el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
3. Filtra la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0


In [49]:
# 1. 
data = {
    'id': [1, 2, 3, 4],
    'vector': [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0], [10.0, 11.0, 12.0]]
}
df = pd.DataFrame(data)

# 2. 
df.at[df[df['id'] == 3].index[0], 'vector'] = [10.0, 11.0, 10.0]

# 3. 
filtered_df = df[df['vector'].apply(lambda x: any(i > 9.0 for i in x))]
filtered_df

,id,vector
2,3,"[10.0, 11.0, 10.0]"
3,4,"[10.0, 11.0, 12.0]"


**Task 4: Embeddings multimodales y búsqueda combinada**

Instrucciones:
1. Crea una tabla con datos de texto e imágenes combinados. Incluye las siguientes columnas:
- texto (Texto).
- imagen (Nombre del archivo de imagen).
- embedding_texto (Vector del texto generado con SentenceTransformer).
- embedding_imagen (Vector de la imagen generado con ResNet18).
2. Realiza una consulta para encontrar los elementos con un texto similar a "La tecnología avanza rápido" y una imagen visualmente similar a un color predominantemente azul.
3. Muestra los resultados combinados ordenados por la menor distancia promedio entre ambos embeddings.


In [55]:
import pandas as pd
import pyarrow as pa
from PIL import Image
from sentence_transformers import SentenceTransformer

import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18

data = [
    {"id": 1, "texto": "La roca de joven", "imagen": "imagen_1.jpg"},
    {"id": 2, "texto": "El perro con lentes", "imagen": "imagen_2.jpg"},
    {"id": 3, "texto": "Señor bean", "imagen": "imagen_3.jpg"}
]

df = pd.DataFrame(data)

text_model = SentenceTransformer('all-MiniLM-L6-v2')
image_model = resnet18(pretrained=True)
image_model.fc = torch.nn.Identity()
image_model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

embeddings_texto = []
embeddings_imagen = []

for _, row in df.iterrows():
    embeddings_texto.append(text_model.encode(row["texto"]).tolist())
    image = Image.open(f'/Users/juancasasmartinez/Documents/ITAM/Fuentes/proyectos_finales/proyecto_lance/notebooks/data/images/{row["imagen"]}').convert("RGB")
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        embeddings_imagen.append(image_model(image_tensor).squeeze().tolist())

df["embedding_texto"] = embeddings_texto
df["embedding_imagen"] = embeddings_imagen

embedding_size_texto = len(embeddings_texto[0])
embedding_size_imagen = len(embeddings_imagen[0])

/Users/juancasasmartinez/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/juancasasmartinez/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/juancasasmartinez/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can a

In [57]:
schema = pa.schema([
    ("id", pa.int64()),
    ("texto", pa.string()),
    ("imagen", pa.string()),
    ("embedding_texto", pa.list_(pa.float32(), list_size=embedding_size_texto)),
    ("embedding_imagen", pa.list_(pa.float32(), list_size=embedding_size_imagen))
])

table = pa.Table.from_pandas(df, schema=schema)

tabla = db.create_table("multimodal_data", schema=schema, mode="overwrite")
tabla.add(table)

[2024-12-03T01:47:06Z WARN  lance::dataset] No existing dataset at /Users/juancasasmartinez/Documents/ITAM/Fuentes/proyectos_finales/proyecto_lance/notebooks/data/ANN/multimodal_data.lance, it will be created


In [62]:
from scipy.spatial.distance import cosine


consulta_texto = "La tecnología avanza rápido"
consulta_imagen = "/Users/juancasasmartinez/Desktop/consulta_azul.jpg" 

embedding_consulta_texto = text_model.encode(consulta_texto).tolist()

consulta_image = Image.open(consulta_imagen).convert("RGB")
consulta_tensor = transform(consulta_image).unsqueeze(0)
with torch.no_grad():
    embedding_consulta_imagen = image_model(consulta_tensor).squeeze().tolist()

resultados_texto = tabla.search(embedding_consulta_texto, "embedding_texto").limit(10).to_list()
resultados_imagen = tabla.search(embedding_consulta_imagen, "embedding_imagen").limit(10).to_list()

resultados_combinados = []
for texto_row in resultados_texto:
    for imagen_row in resultados_imagen:
        if texto_row["id"] == imagen_row["id"]:
            distancia_texto = cosine(embedding_consulta_texto, texto_row["embedding_texto"])
            distancia_imagen = cosine(embedding_consulta_imagen, imagen_row["embedding_imagen"])
            promedio_distancia = (distancia_texto + distancia_imagen) / 2
            
            resultados_combinados.append(
                {
                    "id": texto_row["id"],
                    "texto": texto_row["texto"],
                    "imagen": texto_row["imagen"],
                    "distancia_promedio": promedio_distancia
                }
            )

resultados_combinados.sort(key=lambda x: x["distancia_promedio"])

for res in resultados_combinados:
    print(f"ID: {res['id']}, Texto: {res['texto']}, Imagen: {res['imagen']}, Distancia Promedio: {res['distancia_promedio']:.4f}")


ID: 1, Texto: La roca de joven, Imagen: imagen_1.jpg, Distancia Promedio: 0.5710
ID: 2, Texto: El perro con lentes, Imagen: imagen_2.jpg, Distancia Promedio: 0.5883
ID: 3, Texto: Señor bean, Imagen: imagen_3.jpg, Distancia Promedio: 0.6729
